# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 64 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.20069


extracting tarball to tmp_2503.20069...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']


Found 96 bibliographic references in tmp_2503.20069/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.20165


extracting tarball to tmp_2503.20165...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.20069-b31b1b.svg)](https://arxiv.org/abs/2503.20069) | **TOI-2005b: An Eccentric Warm Jupiter in Spin-Orbit Alignment**  |
|| A. Bieryla, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-03-27*|
|*Comments*| *15 pages, 10 figures, 4 tables*|
|**Abstract**|            We report the discovery and characterization of TOI-2005b, a warm Jupiter on an eccentric (e~0.59), 17.3-day orbit around a V_mag = 9.867 rapidly rotating F-star. The object was detected as a candidate by TESS and the planetary nature of TOI-2005b was then confirmed via a series of ground-based photometric, spectroscopic, and diffraction-limited imaging observations. The planet was found to reside in a low sky-projected stellar obliquity orbit (lambda = 4.8 degrees) via a transit spectroscopic observation using the Magellan MIKE this http URL-2005b is one of a few planets known to have a low-obliquity, high-eccentricity orbit, which may be the result of high-eccentricity coplanar migration. The planet has a periastron equilibrium temperature of ~ 2100 K, similar to some highly irradiated hot Jupiters where atomic metal species have been detected in transmission spectroscopy, and varies by almost 1000 K during its orbit. Future observations of the atmosphere of TOI-2005b can inform us about its radiative timescales thanks to the rapid heating and cooling of the planet.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.20165-b31b1b.svg)](https://arxiv.org/abs/2503.20165) | **Impact of Spectral Resolution on S-index and Its Application to Spectroscopic Surveys**  |
|| H. Han, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-27*|
|*Comments*| *11 pages, 9 figures, ApJ accepted*|
|**Abstract**|            Utilizing the PHOENIX synthetic spectra, we investigated the impact of spectral resolution on the calculation of $S$-indices. We found that for spectra with a resolution lower than $\approx$30,000, it is crucial to calibrate $S$-indices for accurate estimations. This is especially essential for low-resolution spectral observations. We provided calibrations for several ongoing or upcoming spectroscopic surveys such as the LAMOST low-resolution survey, the SEGUE survey, the SDSS-V/BOSS survey, the DESI survey, the MSE survey, and the MUST survey. Using common targets between the HARPS and MWO observations, we established conversions from spectral $S$-indices to the well-known $S_{\rm MWO}$ values, applicable to stars with [Fe/H] values greater than $-$1. These calibrations offer a reliable approach to convert $S$-indices obtained from various spectroscopic surveys into $S_{\rm{MWO}}$ values and can be widely applied in studies on chromospheric activity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.20069/./toi2005.png', 'tmp_2503.20069/./TP.png', 'tmp_2503.20069/./TPlines.png', 'tmp_2503.20069/./stackplot_TOI-2005.png']
copying  tmp_2503.20069/./toi2005.png to _build/html/
copying  tmp_2503.20069/./TP.png to _build/html/
copying  tmp_2503.20069/./TPlines.png to _build/html/
copying  tmp_2503.20069/./stackplot_TOI-2005.png to _build/html/
exported in  _build/html/2503.20069.md
    + _build/html/tmp_2503.20069/./toi2005.png
    + _build/html/tmp_2503.20069/./TP.png
    + _build/html/tmp_2503.20069/./TPlines.png
    + _build/html/tmp_2503.20069/./stackplot_TOI-2005.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bjdtdb}{\ensuremath{\rm{BJD_{TDB}}}}$
$\newcommand{\meh}{\ensuremath{\left[{\rm m}/{\rm H}\right]}}$
$\newcommand{\feh}{\ensuremath{\left[{\rm Fe}/{\rm H}\right]}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\vsini}{\ensuremath{v\sin i}}$
$\newcommand{\vsinistar}{\ensuremath{v\sin I_\star}}$
$\newcommand{\loggunits}{cm s^{-2}}$
$\newcommand{\ecosw}{\ensuremath{e\cos{\omega_*}}}$
$\newcommand{\esinw}{\ensuremath{e\sin{\omega_*}}}$
$\newcommand{\aor}{\ensuremath{a/R_{\star}}}$
$\newcommand{\msun}{\ensuremath{M_\Sun}}$
$\newcommand{\rsun}{\ensuremath{R_\Sun}}$
$\newcommand{\lsun}{\ensuremath{L_\Sun}}$
$\newcommand{\rj}{\ensuremath{R_{\rm Jup}}}$
$\newcommand{\mj}{\ensuremath{M_{\rm Jup}}}$
$\newcommand{\fixme}{\textcolor{red}{[FIXME] }}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\tess}{\emph{TESS}\xspace}$
$\newcommand{\gaia}{\emph{Gaia}\xspace}$
$\newcommand{\tres}{TRES\xspace}$
$\newcommand{\ut}{UT\xspace}$
$\newcommand{\bjd}{BJD\xspace}$
$\newcommand{\ms}{ m s^{-1}}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\tic}{TIC 147660886\xspace}$
$\newcommand{\ticb}{TIC 147660886 b\xspace}$
$\newcommand{\toi}{TOI-2005\xspace}$
$\newcommand{\toib}{TOI-2005 b\xspace}$
$\newcommand{\pmRA}{-25.248\pm0.012}$
$\newcommand{\pmDEC}{8.399\pm0.016}$
$\newcommand{\parallax}{2.991\pm0.017}$
$\newcommand{\vsiniLSD}{111\pm1}$
$\newcommand{\loggfit}{3.946\pm0.035}$
$\newcommand{\tefffit}{7130\pm150}$
$\newcommand{\metfit}{0.01\pm0.08}$
$\newcommand{\mstar}{1.59^{+0.016}_{-0.017}}$
$\newcommand{\rstar}{2.02^{+0.14}_{-0.16}}$
$\newcommand{\lstar}{9.17^{+0.52}_{-0.99}}$
$\newcommand{\age}{1.6\pm0.1}$
$\newcommand{\dist}{328.7^{+4.6}_{-4.2}}$
$\newcommand{\per}{17.305904_{-0.000020}^{+0.000023}}$
$\newcommand{\plrad}{1.07^{+0.06}_{-0.11}}$
$\newcommand{\ecc}{0.597^{+0.097}_{-0.065}}$
$\newcommand{\semimaj}{0.16\pm0.02}$
$\newcommand{\lam}{4.8^{+2.3}_{-2.5}}$
$\newcommand{\CfA}{Center for Astrophysics \textbar  Harvard \& Smithsonian, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\USQ}{University of Southern Queensland, Centre for Astrophysics, West Street, Toowoomba, QLD 4350 Australia}$
$\newcommand{\FlatironCCA}{Center for Computational Astrophysics, Flatiron Institute, 162 Fifth Avenue, New York, NY 10010, USA}$
$\newcommand{\UIUC}{Department of Astronomy, University of Illinois at Urbana-Champaign, Urbana, IL 61801, USA}$
$\newcommand{\PSUAA}{Department of Astronomy \& Astrophysics, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\PSUCEHW}{Center for Exoplanets and Habitable Worlds, 525 Davey Laboratory, The Pennsylvania State University, University Park, PA, 16802, USA}$
$\newcommand{\NOIRLAB}{NSF’s National Optical-Infrared Astronomy Research Laboratory, 950 N. Cherry Avenue, Tucson, AZ 85719, USA}$
$\newcommand{\MITKavli}{Department of Physics and Kavli Institute for Astrophysics and Space Research, Massachusetts Institute of Technology, Cambridge, MA 02139, USA}$
$\newcommand{\DTU}{DTU Space,  Technical University of Denmark, Elektrovej 328, DK-2800 Kgs. Lyngby, Denmark}$
$\newcommand{\CarnegieOBS}{The Observatories of the Carnegie Instution for Science, 813 Santa Barbara Street, Pasadena, CA, 91101, USA}$
$\newcommand{\CarnegieLCO}{Las Campanas Observatory, Carnegie Institution for Science, Colina El Pino, Casilla 601 La Serena, Chile}$
$\newcommand{\CarnegieEPL}{Earth and Planets Laboratory, Carnegie Institution for Science, 5241 Broad Branch Road, NW, Washington, DC 20015, USA}$</div>



<div id="title">

# TOI-2005b: An Eccentric Warm Jupiter in Spin-Orbit Alignment

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.20069-b31b1b.svg)](https://arxiv.org/abs/2503.20069)<mark>Appeared on: 2025-03-27</mark> -  _15 pages, 10 figures, 4 tables_

</div>
<div id="authors">

A. Bieryla, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery and characterization of $\toib$ , a warm Jupiter on an eccentric ( $e\sim0.59$ ), 17.3-day orbit around a $V_\mathrm{mag}$ = 9.867 rapidly rotating F-star. The object was detected as a candidate by TESS and the planetary nature of $\toib$ was then confirmed via a series of ground-based photometric, spectroscopic, and diffraction-limited imaging observations. The planet was found to reside in a low sky-projected stellar obliquity orbit ( $\lambda = $ $\lam$ degrees) via a transit spectroscopic observation using the Magellan MIKE spectrograph. $\toib$ is one of a few planets known to have a low-obliquity, high-eccentricity orbit, which may be the result of high-eccentricity coplanar migration. The planet has a periastron equilibrium temperature of $\sim 2100$ K, similar to some highly irradiated hot Jupiters where atomic metal species have been detected in transmission spectroscopy, and varies by almost 1000 K during its orbit. Future observations of the atmosphere of TOI-2005b can inform us about its radiative timescales thanks to the rapid heating and cooling of the planet.

</div>

<div id="div_fig1">

<img src="tmp_2503.20069/./toi2005.png" alt="Fig9" width="100%"/>

**Figure 9. -** Eccentricity vs. semi-major axis for all confirmed transiting planets between 6--20 Earth radii. Colored circles are planets with measured spin-orbit angles. Blue circles are defined as `aligned' with spin-orbit angles of less than 30 degrees and red circles are defined as `misaligned' with spin-orbit angles greater than 30 degrees. The gray region illustrates planets that are likely undergoing high-eccentricity tidal migration. The upper and lower limits of the track are set by the Roche limit and the tidal circularization timescale  ([Dong, Huang and Zhou 2021]()) . $\toi$b sits on the lower edge of the migration track. The dotted-dashed line presents the theoretical upper limit of eccentricities as a result of planet-planet scattering, assuming a planet with a mass of 0.5 $M_{\rm Jup}$ and a radius of 2 $R_{\rm Jup}$, for illustrative purpose  ([Petrovich, Tremaine and Rafikov 2014]()) .
Data were compiled from the NASA Exoplanet Archive as of October 30, 2024. Stellar obliquity measurements were obtained from [Knudstrup, Albrecht and Winn (2024)]() Table B2 with recent updates. (*fig:ecc_au*)

</div>
<div id="div_fig2">

<img src="tmp_2503.20069/./TP.png" alt="Fig5.1" width="50%"/><img src="tmp_2503.20069/./TPlines.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Temperature as a function of pressure over orbital phase from a 1D timestepping cloud-free atmospheric model generated using \texttt{EGP+}. Due to the presence of TiO and VO in the model atmosphere, at periastron the temperature of the upper atmosphere rises rapidly causing an inversion. (*fig:TP*)

</div>
<div id="div_fig3">

<img src="tmp_2503.20069/./stackplot_TOI-2005.png" alt="Fig1" width="100%"/>

**Figure 1. -** Ground-based and TESS lightcurves normalized and offset relatively in flux and phased in time to the best-fit ephemeris. The best-fit model is shown in black. (*fig:allLCs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.20069"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

408  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

41  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
